In [2]:
import re

import filesTool as ft
import IMDBmedia as IM

IM.imdb['host'] = "localhost"
IM.imdb['user'] = "root"
IM.imdb['pass'] = "root"
IM.imdb['dbname'] = "IMDBmedia"



# widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# IMDB dirs
MovieCast = [
    '/export/Movies'
]

# the dir under this one will be based on media type, then ! title of movie/video/series
ft.Root = '/export/MediaCast'



In [ ]:
def search_title_basics(name, output=False):
    DATA=[]
    for x in IM.search_title_basics(name):
        if output:
            print(x[1] + " (" + str(x[2]) + ") " + "\n\t - " + x[3] + "\n\t" + IMDBurl(x[0]) )
            print("\t"+x[4])
            

def search_title_basics_type(name, MediaType, output=False):
    DATA=[]
    for x in IM.search_title_basics_type(name, MediaType):
        if output:
            print(x[1] + " (" + str(x[2]) + ") " + "\n\t - " + x[3] + "\n\t" + IMDBurl(x[0]) + "\n")
        
        new = []
        new.append(x[0])
        new.append(x[1])
        new.append(x[2])
        new.append(x[3])
        DATA.append(new)
    return DATA            

def search_movie_year(name, year, MediaType, output=False):
    DATA=[]
    for x in IM.search_movie_year(name, year, MediaType):
        if output:
            print(x[1] + " (" + str(x[2]) + ") " + "\n\t - " + x[3] + "\n\t" + IM.IMDBurl(x[0]) + "\n")
        
        new = []
        new.append(x[0])
        new.append(x[1])
        new.append(x[2])
        new.append(x[3])
        DATA.append(new)


In [3]:
        
def check_imdb_man(show, year, file, MediaType):
    SDB=IM.search_video_imdb(show, year, MediaType)
    PICK=[]
    for d in SDB:
        url = IMDBurl( d[0] )
        print("\n"+ d[0] +" "+ d[1] + " (" + str(d[2]) + ") - " + d[3] + "\n\t- " + url )
        val = "|".join(d)
        p = ( d[0], val+"|"+file )
        PICK.append(p)
    interact_manual(use_this_movie_match, match=PICK)

#match is a string, qurey + old file
# need to break with split
def use_this_movie_match(match):
    #print(match)
    d = match.split('|')
    url = IM.IMDBurl(  d[0] )
    old = d[4]
    new = ft.new_file(d[3], str(d[2]), d[1])

    new_dir = ft.new_dir(d[3], str(d[2])) 

    ival=(d[0], d[1], d[2], d[3], new, new_dir, url)

    if ft.install_movie_match(old, new, new_dir):
        IM.add_movie_match(ival)
        #print("mkdir",new_dir)
        #print("mv",old,new)
        print("Added:",new)
    
    
def get_imdb_file_movie(Dir, MediaType):
    Files = []
    Files = ft.scan_files(Dir, Files)
    Files.sort()
    Failed = []
    for x in Files:
        year = ft.get_year(x)
        show = ft.get_show(x)

        SDB=IM.search_video_imdb(show, year, MediaType)
        records=len(SDB)
        #print(SDB,"\n",records,"\n",type(SDB))
        
        if records == 1:
            for d in SDB:
                url = IM.IMDBurl( d[0] )
                print("\n"+d[1] + " (" + str(d[2]) + ") " + "\t-\t" + url + "\n")
                use_this_movie_match("|".join(d)+"|"+x)
                break
        elif records > 1:
            PICK=[]
            for d in SDB:
                url = IM.IMDBurl( d[0] )
                print("\n"+ d[0] +" "+ d[1] + " (" + str(d[2]) + ") - " + d[3] + "\n\t- " + url )
                val = "|".join(d)
                p = ( d[0], val+"|"+x )
                PICK.append(p)
            interact_manual(use_this_movie_match, match=PICK)
        else:
            records = "0"
            
        if records == "0":
            Failed.append({'file': x, 'show' : show, 'year': year})

    for f in Failed:
        print(" => No record in the local Database")
        print("------------------------------------------------")
        print("Show:",f['show'],"\nYear:",f['year'])
        print(" -> skipped",f['file'],"\n - Check the IMDB query link bellow")
        print(IM.IMDBsearch_url( f['show'] ) )
        LIST=IM.media_type( 'list' )
        interact_manual(check_imdb_man, show=f['show'], year=f['year'], file=[ f['file'] ], MediaType=LIST)
        print("________________________________________________")

print("Video Folder lookup for Series, must only have one match to import: ")
interact_manual(get_imdb_file_movie, Dir=MovieCast, MediaType=IM.media_type('ALL'));

Video Folder lookup for Series, must only have one match to import: 


interactive(children=(Dropdown(description='Dir', options=('/export/Movies',), value='/export/Movies'), Dropdo…

In [ ]:
    
def video_move_type_sd2usb(MediaType):
    SDB = IM.search_movie_list_dir_type(ft.SD, MediaType)
    print(len(SDB))
    for x in SDB:
        new_dir = re.sub(ft.SD, ft.USB, os.path.dirname(x[0]) )
        print("mkdir",new_dir)
        new_file = re.sub(ft.SD, ft.USB, x[0])
        print("mv",x[0],new_file,"\n")

        if ft.install_movie_match(x[0],new_file,new_dir):
            print("Moved file to:",new_file,"\n")
            IM.update_movie_list(new_dir, new_file, x[0])
            
def video_move_type_usb2sd(MediaType):
    SDB = IM.search_movie_list_dir_type(ft.USB, MediaType)
    print(len(SDB))
    for x in SDB:
        new_dir = re.sub(ft.USB, ft.SD, os.path.dirname(x[0]) )
        print("mkdir",new_dir)
        new_file = re.sub(ft.USB, ft.SD, x[0])
        print("mv",x[0],new_file,"\n")

        if ft.install_movie_match(x[0],new_file,new_dir):
            print("Moved file to:",new_file,"\n")
            IM.update_movie_list(new_dir, new_file, x[0])